In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date

In [0]:
source = spark.createDataFrame([(1, "Alice","Rochester"),(2, "Bob","Chennai")], ["id", "name","city"])
source.show()
target = spark.createDataFrame([(1, "Alice", "Delhi","2025-06-24","9999-12-31", True)], ["tgt_id", "tgt_name","tgt_city","start_date","end_date","active_flag"])
target.show()



In [0]:
today = date.today().isoformat()
today

In [0]:
joined_df = source.alias("src").join(target.filter(col("active_flag") == True).alias("tgt"), col("src.id") == col("tgt.tgt_id"), "left")

joined_df.show()


In [0]:
changed = joined_df.filter((joined_df.id != joined_df.tgt_id) | (joined_df.name != joined_df.tgt_name))
changed.show()


In [0]:
changed = joined_df.filter((joined_df.city != joined_df.tgt_city) | (joined_df.name != joined_df.tgt_name) | joined_df.tgt_id.isNull())
changed.show()

In [0]:
new_record = changed.filter("tgt_id is NULL").select("id","name","city", lit(today).alias("start_date"), lit("9999-12-31").alias("end_date"),lit(True).alias("active_flag"))
new_record.show()


In [0]:
updated_record = changed.filter("tgt_id is NOT NULL")
updated_record.show()

In [0]:
#updated_record = changed.filter("tgt_id is NOT NULL").select("id","name","city","start_date","end_date","active_flag")
#updated_record.show()


In [0]:
expired_record = updated_record.select("tgt_id", "tgt_name", "tgt_city","start_date", current_date().alias("end_date"),lit(False).alias("active_flag"))
expired_record.show()

In [0]:
scd2_result = new_record.union(expired_record).union(updated_record)
display(scd2_result)